In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers
!pip install bitsandbytes
!pip install pip install accelerate
!pip install sentencepiece

In [ ]:
# imports
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

In [ ]:
MODEL_PATH = "drive/MyDrive/lora-alpaca"  # top level folder

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

In [ ]:
model = PeftModel.from_pretrained(model, MODEL_PATH, local_files_only=True)

In [ ]:
req = input("Anweisung: ")

PROMPT = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request. Write in German."
    "### Instruction:"
    f"{req}"
    "### Response:"
)

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))